# Chapter 7.2 Hierarchical Clustering

In this section, we will learn another clustering algorithm called **hierarchical clustering**. Unlike with $k$-means, the number of clusters does not need to be prespecified in hierarchical clustering. It will learn an entire _hierarchy_ of models: a 1-cluster model, a 2-cluster model, and so on, up to an $n$-cluster model (i.e., where each of the $n$ observations are in their own cluster). This hierarchy has the property that the $(k+1)$-cluster model is obtained by splitting one of the clusters in the $k$-cluster model.

How do we learn a hierarchy of clusters? There are two general approaches:

- **Divisive** ("top-down"): Start with all of the observations in a single cluster. Recursively split clusters into smaller clusters, until every observation is in its own cluster.
- **Agglomerative** ("bottom-up"): Start with each observation in its own cluster. Recursively merge clusters into larger clusters, until every observation is in a single cluster.

In this section, we will focus on _agglomerative_ hierarchical clustering algorithms, which are more commonly used and easier to understand.

## The Algorithm, in Pictures

Suppose we have a dataset consisting of the 5 points shown below.

<img src="hierarchical0.jpg" width="400" />

In agglomerative hierarchical clustering, each point starts out in its own cluster, and we successively merge clusters. The first step is clearly to merge A and B into one cluster, since they are the two closest points.

<img src="hierarchical1.jpg" width="400" />

We now have 4 clusters: {A, B}, {C}, {D}, {E}. What do we merge next? Here things become slightly tricky. So far, we have only defined the distance between two _points_. But now we need a way to measure the distance between a _cluster_ and a _point_, and more generally, the distance between two _clusters_. It turns out that there are many ways to extend a distance metric between points to a distance metric between clusters. These different ways are called **linkages**. The choice of linkage can have a significant influence on the clusters that are obtained. We will return to this point later.

For now, it seems that no matter how you look at it, the "closest" clusters are clearly {C} and {A, B}, so let's merge them.

<img src="hierarchical2.jpg" width="400" />

Next, we merge D and E into one cluster.

<img src="hierarchical3.jpg" width="400" />

And finally, we merge the two clusters into one large cluster containing all 5 observations.

<img src="hierarchical4.jpg" width="400" />

## Dendrograms: A Way to Visualize Hierarchical Clustering

**Dendrograms** (from the Greek word _dendron_, meaning "tree") are a way to visualize a hierarchy of clusters. The individual observations are listed at the tips of the branches. Each U-shaped merger of two branches on this diagram corresponds to two clusters merging. The height at which the branches merge corresponds to the distance between the clusters before they merged.

<img src="dendro1.jpg" width="400" />

Suppose we want two clusters. We would cut the dendrogram at a height where there are exactly two branches remaining. We can see easily from the dendrogram that the resulting clusters are {A, B, C} and {D, E}.

<img src="dendro2.jpg" width="400" />

Similarly, if we wanted four clusters, we would cut the dendrogram at a height where there are exactly four branches remaining. We can see from the dendrogram that the resulting clusters would be {A, B}, {C}, {D}, {E}.

<img src="dendro3.jpg" width="400" />

## More about Linkages

Let's return to the question of how to measure the distance between two clusters. A distance metric for clusters, which we will represent by $D$, is built on top of a distance metric for points, which we will represent by $d$ (which might be, for example, Euclidean distance, taxicab distance, etc.). There are a number of different ways to extend a distance metric for points to a distance metric for clusters. These different ways are called **linkages**. Some common ones are:

- **single linkage:** the distance between the two _closest_ points in the clusters

$$D(A, B) = \min\{ d(a, b): a \in A, b \in B \}$$

- **complete linkage:** the distance between the two _furthest_ points in the clusters

$$D(A, B) = \max\{ d(a, b): a \in A, b \in B \}$$

- **average linkage:** the average of all pairwise distances between points in the clusters

$$D(A, B) = \frac{1}{|A| |B|} \sum_a \sum_b d(a, b) $$

These three linkages are illustrated in the diagrams below.

<img src="linkage1.jpg" width="400" /> <img src="linkage2.jpg" width="400" />

## Hierarchical Clustering in Scikit-Learn

Although hierarchical clustering is intuitive, it is somewhat tricky to implement, so we will not pursue that here. Instead, we will use two professional implementations of hierarchical clustering: one in Scikit-Learn and another in Scipy.

We start with Scikit-Learn because its API is probably familiar to you by now. In Scikit-Learn, you choose the number of clusters upfront. Let's look at how to fit a hierarchical clustering model to the same iris dataset from the previous section.

In [ ]:
%matplotlib inline
import pandas as pd
iris = pd.read_csv("/data301/data/iris.csv")
X_train = iris[["petal length", "petal width"]]

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=3, 
                                affinity="euclidean",
                                linkage="complete")
model.fit(X_train)
clusters = model.labels_

clusters = pd.Series(clusters).map({
    0: "r",
    1: "b",
    2: "y"
})

X_train.plot.scatter(x="petal length", y="petal width", 
                     c=clusters, marker="x", alpha=.5)

The disadvantages of using Scikit-Learn to do hierarchical clustering are: (1) it does not support single linkage (although [it was recently implemented](https://github.com/scikit-learn/scikit-learn/pull/9372) and is now in the dev version), and (2) it does not draw dendrograms. For these features, we need to turn to `scipy`.

## Hierarchical Clustering in Scipy

`scipy` is a Python library for scientific computing. It contains many useful functions for signal processing, optimization, simulation, and clustering. The hierarchical clustering API in `scipy` works as follows:

- First, you create a _linkage matrix_ that encodes the clustering.
- Then, you call `fcluster()` on this linkage matrix to get the cluster assignments or `dendrogram()` to get a plot of the dendrogram. (This function also returns a bunch of other output which you probably do not need; you can suppress the output by adding a semicolon to the end of the line.)

In [ ]:
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram

Z = linkage(X_train, method="complete", metric="euclidean")
clusters = fcluster(Z, 3, criterion="maxclust")

clusters = pd.Series(clusters).map({
    1: "r",
    2: "b",
    3: "y"
})

X_train.plot.scatter(x="petal length", y="petal width", 
                     c=clusters, marker="x", alpha=.5)

In [ ]:
dendrogram(Z);

# Exercises

**Exercise 1.** Fit a hierarchical clustering model to the Titanic passengers dataset (`/data301/data/titanic.csv`). You are free to choose which features to include (but please include both categorical and quantitative features) and the linkage function. Then, choose a number of clusters that seems appropriate. Look at the profiles of the passengers in each cluster. Can you come up with an "interpretation" of each cluster based on the passengers in it?

In [ ]:
# TYPE YOUR CODE HERE

**Exercise 2.** The code below reads in the "two moons" dataset, a synthetic dataset that is used to evaluate clustering algorithms. What clusters do you think hierarchical clustering will find if you use single linkage? What if you use average linkage? Once you have a hypothesis for each type of linkage, test out your hypothesis by fitting the model to this dataset and plotting the resulting clusters.

In [ ]:
# TYPE YOUR CODE HERE
moons = pd.read_csv("/data301/data/two_moons.csv")
moons.plot.scatter(x="x1", y="x2", color="k")

**Exercise 3.** The code below reads in the "satellite" dataset, a synthetic dataset that is used to evaluate clustering algorithms. What clusters do you think hierarchical clustering will find if you use single linkage? What if you use average linkage? Once you have a hypothesis for each type of linkage, test out your hypothesis by fitting the model to this dataset and plotting the resulting clusters.

_Food for thought:_ How do the results here compare to $k$-means?

In [ ]:
# TYPE YOUR CODE HERE
satellite = pd.read_csv("/data301/data/satellite.csv")
satellite.plot.scatter(x="x1", y="x2", color="k")